In [1]:
import os

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def process_bound_path(boundary_path, attr_index, gan_type, condition):
    bound_choices = {}
    if gan_type == 'pggan' and not condition:
        bound_choices[0] = 'pggan_celebahq_eyeglasses_boundary.npy',
        bound_choices[1] = 'pggan_celebahq_gender_boundary.npy'
        bound_choices[2] = 'pggan_celebahq_smile_boundary.npy'
        bound_choices[3] = 'pggan_celebahq_age_boundary.npy'
        bound_choices[5] = 'pggan_celebahq_bald.npy'
        bound_choices[7] = 'pggan_celebahq_narrow_eyes.npy'
        bound_choices[10] = 'pggan_celebahq_blonde_hair.npy'
        bound_choices[13] = 'pggan_celebahq_pale_skin.npy'
    elif gan_type == 'stylegan' and not condition:
        bound_choices[0] = 'stylegan_celebahq_eyeglasses_boundary.npy'
        bound_choices[1] = 'stylegan_celebahq_gender_boundary.npy'
        bound_choices[2] = 'stylegan_celebahq_smile_boundary.npy'
        bound_choices[3] = 'stylegan_celebahq_age_boundary.npy'
        bound_choices[5] = 'stylegan_celebahq_bald.npy'
        bound_choices[7] = 'stylegan_celebahq_narrow_eyes.npy'
        bound_choices[10] = 'stylegan_celebahq_blonde_hair.npy'
        bound_choices[13] = 'stylegan_celebahq_pale_skin.npy'
    elif gan_type == 'pggan' and condition:
        bound_choices = ['pggan_celebahq_eyeglasses_c_all_boundary.npy', 'pggan_celebahq_gender_c_all_boundary.npy',
            'pggan_celebahq_smile_c_all_boundary.npy', 'pggan_celebahq_age_c_all_boundary.npy']
    elif gan_type == 'stylegan' and condition:
        bound_choices = ['stylegan_celebahq_eyeglasses_c_all_boundary.npy', 'stylegan_celebahq_gender_c_all_boundary.npy',
            'stylegan_celebahq_smile_c_all_boundary.npy', 'stylegan_celebahq_age_c_all_boundary.npy']
    else:
        raise ValueError('process_bound_path: unknown gan type.')

    print(boundary_path, bound_choices[attr_index], attr_index)
    return os.path.join(boundary_path, bound_choices[attr_index])

In [4]:
def add(out_dict, dist, error):
    dist = np.linalg.norm(dist)
    error = np.abs(error)
    if dist < 1:
        out_dict[1].append(error)
    elif dist < 2:
        out_dict[2].append(error)
    elif dist < 3:
        out_dict[3].append(error)
    else:
        out_dict[4].append(error)

In [5]:
def summarize(out, out_dict, attr_index, method):
    out = np.mean(out)
    print(f'{method} {attr_index} mean over samples: {out}')
    tmp = []
    for i in range(1, 5):
        print(f'{method} {attr_index} i-{i}: {np.mean(out_dict[i])}')
        tmp.append(np.mean(out_dict[i]))
    print(f'{method} {attr_index} mean over distance: {np.mean(tmp)}')
    
    out = [out]
    out.extend(tmp)
    out.append(np.mean(tmp))
    return out

In [6]:
def fetch_data(attr_index, method):
    fname_pred = f'./predictions/{model}_{method}_{metric}_{attr_index}_{condition}_{steps}.npy'
    fname_noise = f'./predictions/{model}_{method}_{metric}_{attr_index}_{condition}_{steps}_noise.npy'
    fname_grad = f'./predictions/{model}_{method}_{metric}_{attr_index}_{condition}_{steps}_grad.npy'
    
    if method == 'interfacegan':
        noise = np.load(fname_noise).reshape(2500, 40, 512)
        grad = np.load(boundary_path).reshape(-1)
        pred = np.load(fname_pred)
    elif method == 'ours':
        noise = np.load(fname_noise).reshape(2500, 40, 512)
        grad = np.load(fname_grad)
        pred = np.load(fname_pred)
    else:
        raise ValueError('typo?')
        
    return noise, grad, pred

In [7]:
boundary_root = '../boundaries/'
model = 'stylegan'
method = 'interfacegan'
metric = 'taylor_approximation'
attr_index = 2
condition = False
steps = 40
step_size = 8/steps
boundary_path = process_bound_path(boundary_root, attr_index, model, condition)
candidate_index = [0, 1, 2, 3, 7, 10, 13]

../boundaries/ stylegan_celebahq_smile_boundary.npy 2


In [8]:
pd_bs, pd_ours = [], []
for i in candidate_index:
    attr_index = i
    boundary_path = process_bound_path(boundary_root, attr_index, model, condition)
    
    out = []
    out_dict = {1:[], 2:[], 3:[], 4:[]}
    method = 'interfacegan'
    bs_noise, bs_grad, bs_pred = fetch_data(attr_index, method)
    for i_t in range(2500):
        for i_s in range(39):
            grad = bs_grad
            if bs_pred[i_t, 0, attr_index] >= 0.5:
                grad *= -1
            if attr_index == 3:
                grad *= -1

            dist = bs_noise[i_t, i_s+1] - bs_noise[i_t, i_s]
            approx = bs_pred[i_t, i_s, attr_index] + np.dot(dist, step_size * grad)
            error = bs_pred[i_t, i_s+1, attr_index] - approx
            out.append(np.abs(error))
            add(out_dict, bs_noise[i_t, i_s] - bs_noise[i_t, 0], error)
    pd_bs.append(summarize(out, out_dict, attr_index, method))
    
    out = []
    out_dict = {1:[], 2:[], 3:[], 4:[]}
    method = 'ours'
    ours_noise, ours_grad, ours_pred = fetch_data(attr_index, method)
    for i_t in range(2500):
        for i_s in range(39):
            dist = ours_noise[i_t, i_s+1] - ours_noise[i_t, i_s]
            approx = ours_pred[i_t, i_s, attr_index] + np.dot(dist, step_size * ours_grad[i_t, i_s])
            error = ours_pred[i_t, i_s+1, attr_index] - approx
            out.append(np.abs(error))
            add(out_dict, ours_noise[i_t, i_s] - ours_noise[i_t, 0], error)
    pd_ours.append(summarize(out, out_dict, attr_index, method))

../boundaries/ stylegan_celebahq_eyeglasses_boundary.npy 0
interfacegan 0 mean over samples: 1.9574591772606627
interfacegan 0 i-1: 1.7597279619938166
interfacegan 0 i-2: 2.778592498692452
interfacegan 0 i-3: 3.644210036799539
interfacegan 0 i-4: 1.4805114013414746
interfacegan 0 mean over distance: 2.41576047470682
ours 0 mean over samples: 1.885230008993858
ours 0 i-1: 1.6746939155667906
ours 0 i-2: 2.401363200232644
ours 0 i-3: 2.4690528221413084
ours 0 i-4: 1.557353069389228
ours 0 mean over distance: 2.025615751832493
../boundaries/ stylegan_celebahq_gender_boundary.npy 1
interfacegan 1 mean over samples: 2.0530005688305084
interfacegan 1 i-1: 5.701541791946746
interfacegan 1 i-2: 4.045456476244708
interfacegan 1 i-3: 1.7984887292492344
interfacegan 1 i-4: 0.8913239808513329
interfacegan 1 mean over distance: 3.1092027445730053
ours 1 mean over samples: 2.2571902073971577
ours 1 i-1: 4.469476344225325
ours 1 i-2: 3.6941879891482072
ours 1 i-3: 1.790237711204013
ours 1 i-4: 0.81213

In [9]:
pd_bs = pd.DataFrame(data=pd_bs,    # values
              index=np.arange(len(candidate_index)),    # 1st column as index
              columns=['mean_sample', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_dist'])
print(pd_bs)
pd_bs.to_csv('bs.csv')
pd_ours = pd.DataFrame(data=pd_ours,    # values
              index=np.arange(len(candidate_index)),    # 1st column as index
              columns=['mean_sample', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_dist'])
print(pd_ours)
pd_ours.to_csv('ours.csv')

   mean_sample    mean_1    mean_2    mean_3    mean_4  mean_dist
0     1.957459  1.759728  2.778592  3.644210  1.480511   2.415760
1     2.053001  5.701542  4.045456  1.798489  0.891324   3.109203
2     1.264995  1.763724  1.782642  1.693412  0.960799   1.550144
3     1.751448  2.350216  2.433930  2.312301  1.354435   2.112721
4     0.869178  0.826330  0.849569  0.862586  0.885376   0.855965
5     0.662759  0.665751  0.712200  0.708830  0.642024   0.682201
6     0.247367  0.249296  0.252159  0.258272  0.243654   0.250845
   mean_sample    mean_1    mean_2    mean_3    mean_4  mean_dist
0     1.885230  1.674694  2.401363  2.469053  1.557353   2.025616
1     2.257190  4.469476  3.694188  1.790238  0.812140   2.691510
2     1.625398  3.190526  2.391117  1.610935  0.921230   2.028452
3     1.168871  0.968670  1.109406  1.893073  1.284604   1.313938
4     1.501634  1.621598  1.503380  1.620148  1.411352   1.539120
5     1.115860  1.939238  1.905847  1.199622  0.634320   1.419757
6     0.37

In [231]:
bs_noise = np.load(fname_noise).reshape(2500, 40, 512)
bs_bound = np.load(boundary_path).reshape(-1)
bs_pred = np.load(fname_pred)
print(bs_noise.shape, bs_bound.shape, bs_pred.shape)

(2500, 40, 512) (512,) (2500, 40, 16)


In [233]:
out = []
out_dict = {1:[], 2:[], 3:[], 4:[]}
for i_t in range(2500):
    for i_s in range(39):
        # f(x) = f(a) + f'(x-a)(x-a)
        # error = real_logit - init_logit + step_size * grad
        grad = bs_bound
        if bs_pred[i_t, 0, attr_index] >= 0.5:
            grad *= -1
        if attr_index == 3:
            grad *= -1
            
        dist = bs_noise[i_t, i_s+1] - bs_noise[i_t, i_s]
        approx = bs_pred[i_t, i_s, attr_index] + np.dot(dist, step_size * grad)
        error = bs_pred[i_t, i_s+1, attr_index] - approx
        out.append(np.abs(error))
        add(out_dict, bs_noise[i_t, i_s] - bs_noise[i_t, 0], error)
        #print(bs_pred[i_t, i_s+1, attr_index], approx, error)
        #break
    #break

In [234]:
out = np.mean(out)
print(out)
tmp = []
for i in range(1, 5):
    print(len(out_dict[i]))
    print(np.mean(out_dict[i]))
    tmp.append(np.mean(out_dict[i]))
print(np.mean(tmp))

1.2649948448110802
12820
1.763723913593395
12493
1.7826424710713296
12419
1.6934123989861578
59768
0.9607986852949706
1.5501443672364632


In [235]:
method = 'ours'

In [236]:
fname_pred = f'./predictions/{model}_{method}_{metric}_{attr_index}_{condition}_{steps}.npy'
fname_noise = f'./predictions/{model}_{method}_{metric}_{attr_index}_{condition}_{steps}_noise.npy'
fname_grad = f'./predictions/{model}_{method}_{metric}_{attr_index}_{condition}_{steps}_grad.npy'

In [237]:
ours_noise = np.load(fname_noise).reshape(2500, 40, 512)
ours_grad = np.load(fname_grad)
#ours_pred = sigmoid(np.load(fname_pred))
ours_pred = np.load(fname_pred)
print(ours_noise.shape, ours_grad.shape, ours_pred.shape)

(2500, 40, 512) (2500, 40, 512) (2500, 40, 16)


In [238]:
test = ours_noise - ours_noise[:, [0], :]
print(test.shape)
norm = np.linalg.norm(test, axis=-1, ord=2)
print(norm.shape)
print(np.min(norm))
print(np.mean(norm))
print(norm[norm<1].shape)
print(norm[norm>1].shape)
print(norm[norm>2].shape)
print(norm[norm>3].shape)

(2500, 40, 512)
(2500, 40)
0.0
3.4656289285723987
(17337,)
(82663,)
(68904,)
(55540,)


In [239]:
out = []
out_dict = {1:[], 2:[], 3:[], 4:[]}
for i_t in range(2500):
    for i_s in range(39):
        # f(x) = f(a) + f'(x-a)(x-a)
        # error = real_logit - init_logit + step_size * grad
        dist = ours_noise[i_t, i_s+1] - ours_noise[i_t, i_s]
        approx = ours_pred[i_t, i_s, attr_index] + np.dot(dist, step_size * ours_grad[i_t, i_s])
        error = ours_pred[i_t, i_s+1, attr_index] - approx
        out.append(np.abs(error))
        add(out_dict, ours_noise[i_t, i_s] - ours_noise[i_t, 0], error)
        #print(ours_pred[i_t, i_s+1, attr_index], approx, error)
        #break
    #break

In [240]:
out = np.mean(out)
print(out)
tmp = []
for i in range(1, 5):
    print(len(out_dict[i]))
    print(np.mean(out_dict[i]))
    tmp.append(np.mean(out_dict[i]))
print(np.mean(tmp))

1.62539768996798
17294
3.1905255761101676
13745
2.3911174700366833
13350
1.6109352170206142
53111
0.9212297016766653
2.028451991211033
